# Data Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine. 

**Authors:** Johannes Giehl (jfg.eco@cbs.dk), Dana J. Hentschel (djh.eco@cbs.dk)

## General settings

### Packages:

In [218]:
import numpy as np
import pandas as pd
from datetime import timedelta

### Methods:

In [219]:
#method to identify all nodes that should become slack nodes within the network
def check_entries_exist(df, type):
    # Get the entries in Input1 or Input2 that exist in Output1 or Output2
    input1_output = df[df['Input1'].isin(df['Output1']) | df['Input1'].isin(df['Output2'])]['Input1'].tolist()
    input2_output = df[df['Input2'].isin(df['Output1']) | df['Input2'].isin(df['Output2'])]['Input2'].tolist()

    entries_exist = input1_output + input2_output

    # Create a dictionary with True or False for each entry
    entries_exist_status = {entry: entry in entries_exist for entry in df['Input1'].tolist() + df['Input2'].tolist()}

    entries_exist_status_df = pd.DataFrame(entries_exist_status.items())

    entries_exist_status_df = entries_exist_status_df[entries_exist_status_df[0] != '']
    entries_exist_status_df = entries_exist_status_df.rename(columns={0: type})
    entries_exist_status_df = entries_exist_status_df.rename(columns={1: 'node_slack_penalty'})
    
    return entries_exist_status_df

#check if a value exist once 
def check_exclusively_once(val):
    count = sum(1 for col in inputs_outputs if val in occurrences[col] and occurrences[col][val] == 1)
    return count == 1

### Base parameters

In [220]:
#define year and create time stamp
year = 2018   #change to desired year
start_date = pd.Timestamp(str(year) + '-01-01 00:00:00')
end_date = pd.Timestamp(str(year) + '-12-31 23:00:00')
#set area
area = 'DK1'   #change to desired area
#scenario
scenario = 'Base'
#frequency model
frequency = '1h'
#model name
model_name = 'toy'
#temporal block
temporal_block = 'hourly'
#stochastics
stochastic_scenario = "realisation"
stochastic_structure = "deterministic"
#required reports
report_name = 'Report'
reports = ['unit_flow', 'connection_flow', 'node_state', 'total_costs']

### File paths

In [221]:
#set path to correct folders
#input data
excel_file_path = '../Input_data/Input_raw/'
#prepared input data
output_file_path = '../Input_data/input_prepared/'

In [222]:
#set name of the relevant files

model_parameters = 'Full_Excel.xlsx'
Model_structure_file = 'Model_Data_Base.xlsx'

PV_data_availabilityfactors = 'PV_availability_factors_Kasso_' + str(year) + '.xlsx'
data_powerprices = 'Day_ahead_prices_' + str(year) + '.xlsx'

#output file
output_file_name = 'Maersk_Example_Input_prepared.xlsx'


## Workflow of the data preparation

### General parameters

In [223]:
#date index
date_index = pd.date_range(start=start_date, end=end_date, freq='H')
formatted_dates = date_index.strftime('%Y-%m-%dT%H:%M:%S')
df_formatted_dates = pd.DataFrame(formatted_dates, columns=['DateTime'])

df_time = pd.DataFrame(df_formatted_dates)

### Data import

In [224]:
#read in the model structure from a new excel
df_model_units = pd.read_excel(excel_file_path + Model_structure_file, sheet_name='Units', index_col=None)
df_model_connections = pd.read_excel(excel_file_path + Model_structure_file, sheet_name='Connections', index_col=None)
df_model_storages = pd.read_excel(excel_file_path + Model_structure_file, sheet_name='Storages', index_col=None)
#Object-to-node
df_object__to_node_values = pd.read_excel(excel_file_path + model_parameters, sheet_name='Object__to_node')
#Object-from-node
df_object__from_node_values = pd.read_excel(excel_file_path + model_parameters, sheet_name='Object__from_node')
#Object-node-node
df_object__node_node_values = pd.read_excel(excel_file_path + model_parameters, sheet_name='Object__node_node')
#Variable efficiency
df_variable_efficiency = pd.read_excel(excel_file_path + model_parameters, sheet_name='Variable_Eff')
#Availability factor
df_PV_availabilityfactors_values = pd.read_excel(excel_file_path+PV_data_availabilityfactors, skiprows=2, usecols=[0,1,2,3,4,5])
#Power prices
df_powerprices_total_values = pd.read_excel(excel_file_path+data_powerprices)
#only extracting the prices from our earlier defined area
df_powerprices_values = df_powerprices_total_values[df_powerprices_total_values['PriceArea'] == area]
df_powerprices_values = df_powerprices_values.reset_index(drop=True)
#Storage
df_storage_raw = pd.read_excel(excel_file_path + model_parameters, sheet_name='Storage')

FileNotFoundError: [Errno 2] No such file or directory: '../Input_data/Input_raw/PV_availability_factors_Kasso_2018.xlsx'

### Adjustments

#### Adjust base elements:

In [ ]:
#Create a dataframe for units
df_units = df_model_units[['Unit']].copy()
df_units['Category'] = 'unit'
df_units = df_units.rename(columns={'Unit': 'Object_Name'})

#Create a dataframe for connections
df_connections = df_model_connections[['Connection']].copy()
df_connections['Category'] = 'connection'
df_connections = df_connections.rename(columns={'Connection': 'Object_Name'})

#create a list of nodes of the model
U_input1_nodes = df_model_units['Input1'].tolist()
U_input2_nodes = df_model_units['Input2'].tolist()
U_output1_nodes = df_model_units['Output1'].tolist()
U_output2_nodes = df_model_units['Output2'].tolist()
C_input1_nodes = df_model_connections['Input1'].tolist()
C_input2_nodes = df_model_connections['Input2'].tolist()
C_output1_nodes = df_model_connections['Output1'].tolist()
C_output2_nodes = df_model_connections['Output2'].tolist()

# Combine values from both columns into a single list
All_nodes_list = U_input1_nodes+U_input2_nodes+U_output1_nodes+U_output2_nodes+C_input1_nodes+C_input2_nodes+C_output1_nodes+C_output2_nodes

# Create a list with unique entries
unique_nodes_list = list(set(All_nodes_list))

#Create a dataframe for nodes
df_nodes = pd.DataFrame(unique_nodes_list, columns=['Object_Name'])
df_nodes['Category'] = 'node'
df_nodes = df_nodes.dropna()

#Combined dataframe
df_definition = pd.concat([df_units, df_nodes, df_connections], ignore_index=True)

In [ ]:
#Balance_type
inputs_outputs = ['Input1', 'Input2', 'Output1', 'Output2']
df_combined = pd.concat([df_model_units, df_model_connections])
df_combined = df_combined.reset_index(drop=True)
occurrences = {col: df_combined[col].value_counts() for col in inputs_outputs}
#use the check_esclusively_once method
df_nodes['balance_type'] = df_nodes['Object_Name'].apply(check_exclusively_once)
df_nodes['balance_type'] = df_nodes['balance_type'].replace({True: 'balance_type_none', False: 'balance_type_node'})
#manually change balance_type of Power_Wholesale to balance_type_none
df_nodes.at[4, 'balance_type'] = 'balance_type_none'
#show df head for control
df_nodes.head()

In [ ]:
#add has_state_node_state_cap and frac_state_loss
df_storages_short = df_model_storages.loc[:, ['Storage', 'has_state', 'node_state_cap', 'frac_state_loss']].rename(columns={'Storage': 'Object_Name'})
df_nodes = pd.merge(df_nodes, df_storages_short, on='Object_Name', how='left')
df_nodes.head()

In [ ]:
#create a dataframe with all nodes that should have a slack based on the unit input and outputs
nodes_for_slack_df = check_entries_exist(df_model_units, 'node')
#merge the information into the prepared data frame
merged_df = pd.merge(df_nodes, nodes_for_slack_df, left_on='Object_Name', right_on='node', how='left')
merged_df = merged_df.drop(columns=['node'])
merged_df['node_slack_penalty'] = merged_df['node_slack_penalty'].replace({True: 100000, False: ''})
#create a dataframe with all nodes that should have a slack based on the connection input and outputs
nodes_for_slack_df2 = check_entries_exist(df_model_connections, 'connection')
#merge the information into the prepared data frame
merged_df2 = pd.merge(df_nodes, nodes_for_slack_df2, left_on='Object_Name', right_on='connection', how='left')
merged_df2 = merged_df2.drop(columns=['connection'])
merged_df2['node_slack_penalty'] = merged_df2['node_slack_penalty'].replace({True: 100000, False: ''})
#link the information about the slack of both data frames
merged_df2['node_slack_penalty'] = merged_df['node_slack_penalty'].combine_first(merged_df2['node_slack_penalty'])
#clean the information that only nodes with 'balance_type_node' have a penalty
merged_df2.loc[merged_df2['balance_type'] == 'balance_type_none', 'node_slack_penalty'] = ''
#add the information into the df_nodes
df_nodes['node_slack_penalty'] = merged_df2['node_slack_penalty']

In [ ]:
#Add parameters for connections
df_connections = df_model_connections.loc[:, ['Connection', 'Connection_type']]
insert_index=1
parameter_name = ['connection_type']*len(df_connections)
df_connections.insert(insert_index, 'Parameter_name', parameter_name)
#show table for control
df_connections

#### Times series:

In [ ]:
#adjust PV columns names
df_PV_availabilityfactors_values.rename(columns={'time': 'time [UTC]', 
                                                 'local_time': 'time [' + area + ']',
                                                 'electricity': 'unit_availability_factor'}, inplace=True)
df_powerprices_values.rename(columns={'HourUTC': 'time [UTC]', 
                                         'HourDK': 'time [' + area + ']'}, inplace=True)

### Fitting data into format

#### Relationships:

Object__node_node:

In [225]:
#Define which columns to check
columns_In_In_Unit = ['Unit', 'Input1', 'Input2']
columns_In_Out_Unit = ['Unit', 'Input1', 'Output1']
columns_Out_Out_Unit = ['Unit', 'Output1', 'Output2']
columns_In_In_Connection = ['Connection', 'Input1', 'Input2']
columns_In_Out_Connection = ['Connection', 'Input1', 'Output1']
columns_Out_Out_Connection = ['Connection', 'Output1', 'Output2']

####UNITS#####
#create list of tuples with values of cells + fix_ratio_XXX_XXX
values_in_in_units = [('unit__node__node', 'unit', row[columns_In_In_Unit[0]], row[columns_In_In_Unit[1]], row[columns_In_In_Unit[2]], 'fix_ratio_in_in_unit_flow', row['Relation_In_In']) 
                      if not pd.isnull(row[columns_In_In_Unit]).any() else (np.nan, np.nan, None) for _, row in df_model_units.iterrows() 
                      if not pd.isnull(row[columns_In_In_Unit]).any()]
values_in_out_units = [('unit__node__node', 'unit', row[columns_In_Out_Unit[0]], row[columns_In_Out_Unit[1]], row[columns_In_Out_Unit[2]], 'fix_ratio_in_in_unit_flow', row['Relation_In_Out']) 
                      if not pd.isnull(row[columns_In_Out_Unit]).any() else (np.nan, np.nan, None) for _, row in df_model_units.iterrows() 
                      if not pd.isnull(row[columns_In_Out_Unit]).any()]
values_out_out_units = [('unit__node__node', 'unit', row[columns_Out_Out_Unit[0]], row[columns_Out_Out_Unit[1]], row[columns_Out_Out_Unit[2]], 'fix_ratio_out_out_unit_flow', row['Relation_Out_Out']) 
                      if not pd.isnull(row[columns_Out_Out_Unit]).any() else (np.nan, np.nan, None) for _, row in df_model_units.iterrows() 
                      if not pd.isnull(row[columns_Out_Out_Unit]).any()]

df_fix_ratio_in_in_units = pd.DataFrame(values_in_in_units, columns=['relationship', 'Object_class', 'Object', 'Node1', 'Node2', 'fix_ratio', 'value'])
df_fix_ratio_in_out_units = pd.DataFrame(values_in_out_units, columns=['relationship', 'Object_class', 'Object', 'Node1', 'Node2', 'fix_ratio', 'value'])
df_fix_ratio_out_out_units = pd.DataFrame(values_out_out_units, columns=['relationship', 'Object_class', 'Object', 'Node1', 'Node2', 'fix_ratio', 'value'])

####CONNECTIONS
values_in_in_connections = [('connection__node__node', 'connection', row[columns_In_In_Connection[0]], row[columns_In_In_Connection[1]], row[columns_In_In_Connection[2]], 'fix_ratio_in_in_connection_flow', row['Relation_In_In']) 
                            if not pd.isnull(row[columns_In_In_Connection]).any() else (np.nan, np.nan, None) for _, row in df_model_connections.iterrows() 
                            if not pd.isnull(row[columns_In_In_Connection]).any()]
values_in_out_connections = [('connection__node__node', 'connection', row[columns_In_Out_Connection[0]], row[columns_In_Out_Connection[1]], row[columns_In_Out_Connection[2]], 'fix_ratio_in_out_connection_flow', row['Relation_In_Out']) 
                             if not pd.isnull(row[columns_In_Out_Connection]).any() else (np.nan, np.nan, None) for _, row in df_model_connections.iterrows() 
                             if not pd.isnull(row[columns_In_Out_Connection]).any()]
values_out_out_connections = [('connection__node__node', 'connection', row[columns_Out_Out_Connection[0]], row[columns_Out_Out_Connection[1]], row[columns_Out_Out_Connection[2]], 'fix_ratio_out_out_connection_flow', row['Relation_Out_Out']) 
                              if not pd.isnull(row[columns_Out_Out_Connection]).any() else (np.nan, np.nan, None) for _, row in df_model_connections.iterrows() 
                              if not pd.isnull(row[columns_Out_Out_Connection]).any()]

df_fix_ratio_in_in_connections = pd.DataFrame(values_in_in_connections, columns=['relationship', 'Object_class', 'Object', 'Node1', 'Node2', 'fix_ratio', 'value'])
df_fix_ratio_in_out_connections = pd.DataFrame(values_in_out_connections, columns=['relationship', 'Object_class', 'Object', 'Node1', 'Node2', 'fix_ratio', 'value'])
df_fix_ratio_out_out_connections = pd.DataFrame(values_out_out_connections, columns=['relationship', 'Object_class', 'Object', 'Node1', 'Node2', 'fix_ratio', 'value'])

#create Object_node_node
df_object_node_node = pd.concat([df_fix_ratio_in_in_units, df_fix_ratio_in_out_units, df_fix_ratio_out_out_units, 
                                df_fix_ratio_in_in_connections, df_fix_ratio_in_out_connections, df_fix_ratio_out_out_connections])
df_object_node_node = df_object_node_node.reset_index(drop=True)

#show df head for control
df_object_node_node.head()

,relationship,Object_class,Object,Node1,Node2,fix_ratio,value
0,unit__node__node,unit,Electrolyzer,Power_Kasso,Water,fix_ratio_in_in_unit_flow,NaN
1,unit__node__node,unit,Methanol_Reactor,Hydrogen_Kasso,Vaporized_Carbon_Dioxide,fix_ratio_in_in_unit_flow,1.0
2,unit__node__node,unit,Electrolyzer,Power_Kasso,Hydrogen_Kasso,fix_ratio_in_in_unit_flow,2.0
3,unit__node__node,unit,CO2_Vaporizer,Carbon_Dioxide,Vaporized_Carbon_Dioxide,fix_ratio_in_in_unit_flow,1.0
4,unit__node__node,unit,Destilation_Tower,Raw_Methanol,E-Methanol_Kasso,fix_ratio_in_in_unit_flow,1.0


Object__from/to_node:

In [ ]:
# Initialize an empty list to store the transformed data
unit_relation_parameter_data = []

# Iterate over each row in the DataFrame
for index, row in df_model_units.iterrows():
    unit = row['Unit']

    # Iterate over Input and Output columns
    for i in range(1, 3):
        input_col = f'Input{i}'
        output_col = f'Output{i}'
        cap_input_col = f'Cap_{input_col}_existing'
        cap_output_col = f'Cap_{output_col}_existing'

        # Check for Input columns
        input_value = row[input_col]
        input_capacity = row[cap_input_col]

        if pd.notna(input_value):
            unit_relation_parameter_data.append({
                'relationship_class_name': 'unit__from_node',
                'object_class': 'unit',
                'object_name': unit,
                'node': input_value,
                'parameter_name': 'unit_capacity' if pd.notna(input_capacity) else '',
                'value': input_capacity if pd.notna(input_capacity) else ''
            })

        # Check for Output columns
        output_value = row[output_col]
        output_capacity = row[cap_output_col]

        if pd.notna(output_value):
            unit_relation_parameter_data.append({
                'relationship_class_name': 'unit__to_node',
                'object_class': 'unit',
                'object_name': unit,
                'node': output_value,
                'parameter_name': 'unit_capacity' if pd.notna(output_capacity) else '',
                'value': output_capacity if pd.notna(output_capacity) else ''
            })

# Create a new DataFrame from the transformed data
df_unit_relation_parameter_data = pd.DataFrame(unit_relation_parameter_data)

In [ ]:
# Initialize an empty list to store the transformed data
connection_relation_parameter_data = []

# Iterate over each row in the DataFrame
for index, row in df_model_connections.iterrows():
    connection = row['Connection']

    # Iterate over Input and Output columns
    for i in range(1, 3):
        input_col = f'Input{i}'
        output_col = f'Output{i}'
        cap_input_col = f'Cap_{input_col}_existing'
        cap_output_col = f'Cap_{output_col}_existing'

        # Check for Input columns
        input_value = row[input_col]
        input_capacity = row[cap_input_col]

        if pd.notna(input_value):
            connection_relation_parameter_data.append({
                'relationship_class_name': 'connection__from_node',
                'object_class': 'connection',
                'object_name': connection,
                'node': input_value,
                'parameter_name': 'connection_capacity' if pd.notna(input_capacity) else '',
                'value': input_capacity if pd.notna(input_capacity) else ''
            })

        # Check for Output columns
        output_value = row[output_col]
        output_capacity = row[cap_output_col]

        if pd.notna(output_value):
            connection_relation_parameter_data.append({
                'relationship_class_name': 'connection__to_node',
                'object_class': 'connection',
                'object_name': connection,
                'node': output_value,
                'parameter_name': 'connection_capacity' if pd.notna(output_capacity) else '',
                'value': output_capacity if pd.notna(output_capacity) else ''
            })

# Create a new DataFrame from the transformed data
df_connection_relation_parameter_data = pd.DataFrame(connection_relation_parameter_data)

In [ ]:
#Create combined DataFrame:
df_object__node = pd.concat([df_unit_relation_parameter_data, df_connection_relation_parameter_data])
df_object__node = df_object__node.reset_index(drop=True)

#show df head for control
df_object__node.head()

#### Demand and Renewables Availability:

In [ ]:
#create table headers and relations
column_names_1 = {'DateTime '+area: [None, None],
                'Hydrogen_Kasso': ['node','demand'], 
                'E-Methanol_Kasso': ['node','demand'], 
                'Solar_Plant_Kasso': ['unit','unit_availability_factor']}
df_blank_table_1 = pd.DataFrame(column_names_1, index=None)
#add values
df_temp_1 = pd.DataFrame(columns=['DateTime ' + area, 'Hydrogen_Kasso', 'E-Methanol_Kasso', 'Solar_Plant_Kasso'])

df_temp_1['DateTime '+area] = df_time
df_temp_1['Hydrogen_Kasso'] = 0
df_temp_1['E-Methanol_Kasso'] = 25
df_temp_1['Solar_Plant_Kasso'] = df_PV_availabilityfactors_values['unit_availability_factor']

df_table_1 = pd.concat([df_blank_table_1, df_temp_1])
#show table head for control
df_table_1.head()

#### Energy prices:

In [ ]:
column_names_2 = {'DateTime ' + area: ['relationship class','connection','node','parameter name'],
                'Power_Wholesale_In': ['connection__from_node','power_line_Wholesale_Kasso','Power_Wholesale','connection_flow_cost'], 
                'Power_Wholesale_Out': ['connection__to_node','power_line_Wholesale_Kasso','Power_Wholesale','connection_flow_cost'], 
                'District_Heating': ['connection__to_node','pipeline_District_Heating','District_Heating','connection_flow_cost']}
df_blank_table_2 = pd.DataFrame(column_names_2, index=None)
df_temp_2 = pd.DataFrame(columns=['DateTime ' + area, 'Power_Wholesale_In', 'Power_Wholesale_Out', 'District_Heating'])

df_temp_2['DateTime ' + area] = df_time
df_temp_2['Power_Wholesale_In'] = df_powerprices_values['SpotPriceEUR']
df_temp_2['Power_Wholesale_Out'] = -1 * df_powerprices_values['SpotPriceEUR']
df_temp_2['District_Heating'] = -1

df_table_2 = pd.concat([df_blank_table_2, df_temp_2], ignore_index=True)
#show table head for control
df_table_2.head()

#### Time Series Storage:

In [ ]:
#date index
before = start_date-timedelta(hours=1)
date_index_beginning = pd.date_range(start=before, end=start_date, freq='H')
formatted_beginning = date_index_beginning.strftime('%Y-%m-%dT%H:%M:%S')
df_formatted_beginning = pd.DataFrame(formatted_beginning, columns=['DateTime'])
df_time_beginning = pd.DataFrame(df_formatted_beginning)
#add one blank row
new_row = pd.Series([])
df_time_beginning = pd.concat([pd.DataFrame([new_row]), df_time_beginning]).reset_index(drop=True)

#concat raw data with time index
df_storage = pd.concat([df_time_beginning, df_storage_raw], axis=1)
#show table head for control
df_storage.head()

In [ ]:
#date index
before = start_date-timedelta(hours=1)
date_index_beginning = pd.date_range(start=before, end=start_date, freq='H')
formatted_beginning = date_index_beginning.strftime('%Y-%m-%dT%H:%M:%S')
df_formatted_beginning = pd.DataFrame(formatted_beginning, columns=['DateTime'])
df_time_beginning = pd.DataFrame(df_formatted_beginning)
#add one blank row
new_row = pd.Series([])
df_time_beginning = pd.concat([pd.DataFrame([new_row]), df_time_beginning]).reset_index(drop=True)

#Create storage
selected_columns = ['Storage', 'node_state', 'value_before', 'value_start']
table_B = df_model_storages[selected_columns]
df_storage = table_B.transpose()
df_storage.columns = df_storage.iloc[0]
df_storage = df_storage[1:]

#concat
df_storage = pd.concat([df_time_beginning.reset_index(drop=True), df_storage.reset_index(drop=True)], axis=1)
#show table for control
df_storage

#### Model relations:

In [ ]:
#Writing definition of model components
column_names_model_components = {'Object_class_name':['model','temporal_block','stochastic_scenario', 'stochastic_structure', 'report'],
                      'Object_name': [model_name, temporal_block, stochastic_scenario, stochastic_structure, report_name]}
df_model_components = pd.DataFrame(column_names_model_components, index=None)
#outputs:
df_outputs = pd.DataFrame({
    'Object_class_name': ['output']*len(reports),
    'Object_name': reports
})
df_model_components = pd.concat([df_model_components, df_outputs], axis=0)
df_model_components = df_model_components.reset_index(drop=True)

In [ ]:
#reports
column_names_model_structure = {'Object_class_name':['model','temporal_block','stochastic_scenario', 'stochastic_structure', 'report'],
                      'Object_name': [model_name, temporal_block, stochastic_scenario, stochastic_structure, report_name]}
df_reports = pd.DataFrame({
    'Relationship_class_name': ['report__output']*len(reports),
    'Object_class_name_1': ['report']*len(reports),
    'Object_class_name_2': ['output']*len(reports),
    'Object_name_1': [report_name]*len(reports),
    'Object_name_2': reports
})
#everything else
df_model_struc = pd.DataFrame({
    'Relationship_class_name': ['model__temporal_block','model__default_temporal_block', 
                                'model__stochastic_structure', 'model__default_stochastic_structure',
                                'stochastic_structure__stochastic_scenario', 'model__report'],
    'Object_class_name_1': ['model','model','model','model','stochastic_structure','model'],
    'Object_class_name_2': ['temporal_block', 'temporal_block','stochastic_structure','stochastic_structure', 'stochastic_scenario','report'],
    'Object_name_1': [model_name, model_name,model_name,model_name,stochastic_structure, model_name],
    'Object_name_2': [temporal_block, temporal_block, stochastic_structure, stochastic_structure, stochastic_scenario, report_name]
})
df_model_relations = pd.concat([df_model_struc, df_reports], axis=0)
df_model_relations = df_model_relations.reset_index(drop=True)
#show table head for control
df_model_relations

#### Model:

In [ ]:
### take first 3 columns and add another table with "Alternative", "Value"
column_names_model = {'Object_class_name':['model','model','temporal_block'],
                      'Object_name': [model_name, model_name, temporal_block],
                      'Parameter':['model_start','model_end','resolution'],
                      'Alternative': [scenario, scenario, scenario],
                      'Value': ['{"type": "date_time", "data": "'+df_time.iloc[0]['DateTime']+'"}',
                          '{"type": "date_time", "data": "'+df_time.iloc[-1]['DateTime']+'"}', 
                          '{"type":"duration", "data": "'+frequency+'"}']}
df_model = pd.DataFrame(column_names_model, index=None)
#show table head for control
df_model.head()

### Creating one combined excel and export

1. Definition of objects (df_definition)
2. Balance type + node slack penalty + sonstiges (df_nodes)   NODE SLACK PENALTIES DO NOT WORK YET
3. Parameter connections (df_connections)
4. Relationship classes (df_object__node)
5. Object-node-node (df_object__node_node)
6. Variable_Eff (from Full_Excel)
7. Time_Series_Storage (df_storage)
8. Demand (df_table_1)
9. Energy prices (df_table_2)
10. Model components (df_model_components
11. Model relationships (df_model_relations)
12. Model (df_model)

In [ ]:
#create the prepared input excel for the use in SpineToolbox
with pd.ExcelWriter(output_file_path + output_file_name) as writer:
    df_definition.to_excel(writer, sheet_name='Definition', index=False)
    df_nodes.to_excel(writer, sheet_name='Nodes', index=False)
    df_connections.to_excel(writer, sheet_name='Connections', index=False)
    df_object__node.to_excel(writer, sheet_name='Object__to_from_node', index=False)
    df_object_node_node.to_excel(writer, sheet_name='Object__node_node', index=False)
    df_variable_efficiency.to_excel(writer, sheet_name='Variable_Eff', index=False)
    df_storage.to_excel(writer, sheet_name='Time_series_storage', index=False)
    df_table_1.to_excel(writer, sheet_name='Demand', index=False)
    df_table_2.to_excel(writer, sheet_name='Energy_prices', index=False)
    df_model_components.to_excel(writer, sheet_name='Model_components', index=False)
    df_model_relations.to_excel(writer, sheet_name='Model_relations', index=False)
    df_model.to_excel(writer, sheet_name='Model', index=False)